In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from transformers import BertConfig, AutoTokenizer, TFBertModel, BertTokenizer, TFBertForSequenceClassification, BertModel
#import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd

#import functions
from utils.registry import load_model
from utils.data_preproc import cleaning_text

#import fast_api
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

#import packages
from transformers import  BertTokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import Tokenizer
from transformers import BertConfig, AutoTokenizer, TFBertModel, BertTokenizer, TFBertForSequenceClassification, BertModel
import nltk



ModuleNotFoundError: No module named 'fastapi'

In [2]:
data=pd.read_csv("../data/processed_dataset_v1.csv",nrows=10000)

data['text_processed'] = data['text_processed'].astype(str)
data['text_processed'] = data['text_processed'].str.strip()

In [16]:
# Prepare X and y
X = data['text_processed']
y = data['offensive']

# Création des test et des trains
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1)

X_train=X_train.tolist()
X_test = X_test.tolist()

In [ ]:
# Initializing a BERT bert-base-uncased style configuration

configuration = BertConfig(vocab_size=30000,hidden_size=768,hidden_act='gelu')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer=keras.optimizers.Adam(learning_rate=0.9)
model.compile(optimizer=optimizer, metrics=["accuracy"])

In [ ]:
# Initializing a BERT bert-base-uncased style configuration

#configuration 
config = BertConfig.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',from_pt=True, config=config)
optimizer=keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, metrics=["accuracy"])

In [ ]:
model.bert.trainable = False
model.summary()

In [ ]:
X_train_tokenized = tokenizer(X_train, return_tensors='tf', padding=True, truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors='tf', padding=True, truncation=True)


In [ ]:
X_train_tokenized = dict(X_train_tokenized)
X_test_tokenized = dict(X_test_tokenized)

In [ ]:
es=EarlyStopping(patience=2, restore_best_weights=True, monitor='loss')

In [ ]:
model.fit(X_train_tokenized,y_train, batch_size=32, epochs=10, callbacks=[es])

In [17]:
# Initializing a BERT tiny model style configuration

tokenizer_tiny = BertTokenizer.from_pretrained('prajjwal1/bert-tiny')
model_tiny = TFBertForSequenceClassification.from_pretrained('prajjwal1/bert-tiny',from_pt=True)
optimizer=keras.optimizers.Adam(learning_rate=0.0001)
model_tiny.compile(optimizer=optimizer, metrics=["accuracy"])


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model_tiny.bert.trainable = False
model_tiny.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  4385920   
                                                                 
 dropout_31 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  258       
                                                                 
Total params: 4386178 (16.73 MB)
Trainable params: 258 (1.01 KB)
Non-trainable params: 4385920 (16.73 MB)
_________________________________________________________________


In [19]:
X_train_tokenized = tokenizer_tiny(X_train, return_tensors='tf', padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer_tiny(X_test, return_tensors='tf', padding=True, truncation=True,max_length=512)

X_train_tokenized = dict(X_train_tokenized)
X_test_tokenized = dict(X_test_tokenized)
es=EarlyStopping(patience=2, restore_best_weights=True, monitor='loss')


In [20]:
model_tiny.fit(X_train_tokenized,y_train, batch_size=32, epochs=10, callbacks=[es],validation_split=.2)

Epoch 1/10
175/175 [==============================] - 33s 162ms/step - loss: 0.6687 - accuracy: 0.5941 - val_loss: 0.6358 - val_accuracy: 0.6300
Epoch 2/10
175/175 [==============================] - 29s 164ms/step - loss: 0.5831 - accuracy: 0.6946 - val_loss: 0.5619 - val_accuracy: 0.7143
Epoch 3/10
175/175 [==============================] - 29s 167ms/step - loss: 0.5093 - accuracy: 0.7591 - val_loss: 0.5465 - val_accuracy: 0.7271
Epoch 4/10
175/175 [==============================] - 29s 163ms/step - loss: 0.4446 - accuracy: 0.7957 - val_loss: 0.5793 - val_accuracy: 0.7279
Epoch 5/10
175/175 [==============================] - 28s 161ms/step - loss: 0.3914 - accuracy: 0.8207 - val_loss: 0.5631 - val_accuracy: 0.7193
Epoch 6/10
175/175 [==============================] - 28s 161ms/step - loss: 0.3406 - accuracy: 0.8552 - val_loss: 0.6461 - val_accuracy: 0.7136
Epoch 7/10
175/175 [==============================] - 28s 161ms/step - loss: 0.3021 - accuracy: 0.8732 - val_loss: 0.6600 - val_ac

KeyboardInterrupt: 

In [4]:
# Initializing a BERT mini model style configuration

#configuration 
config = BertConfig.from_pretrained('prajjwal1/bert-mini')
model = TFBertForSequenceClassification(config=config)
model.build()
model.load_weights("../Model/weights-bert.h5")

In [5]:
X = "black people should die"

In [21]:
def predict_binary(X_pred: str):

    #preprocessing
    X_pred = str(X_pred)
    X_pred = cleaning_text(X_pred)
    print("✅ 1. data cleaned")

    #tokenizer_bert
    tokenizer_mini = BertTokenizer.from_pretrained('prajjwal1/bert-mini')
    X_pred = tokenizer_mini(X_pred, return_tensors='tf', padding=True, truncation=True, max_length=512)
    X_pred = dict(X_pred)
    print("✅ 2. data tokenized")

    config = BertConfig.from_pretrained('prajjwal1/bert-mini')
    model = TFBertForSequenceClassification(config=config)
    model.build()
    model.load_weights("bert_binary.h5")
    print("✅ 3. model loaded")

    y_pred = model.predict(X_pred)
    print("✅ 4. prediction made")
    print(y_pred)

    return y_pred

In [20]:
predict_binary(X)

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/irismarechal/nltk_data'
    - '/Users/irismarechal/.pyenv/versions/3.10.6/envs/moder_ia/nltk_data'
    - '/Users/irismarechal/.pyenv/versions/3.10.6/envs/moder_ia/share/nltk_data'
    - '/Users/irismarechal/.pyenv/versions/3.10.6/envs/moder_ia/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [22]:
model_tiny.bert.trainable = True
model_tiny.summary()

Model: "tf_bert_for_sequence_classification_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  4385920   
                                                                 
 dropout_31 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  258       
                                                                 
Total params: 4386178 (16.73 MB)
Trainable params: 4386178 (16.73 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
X_train_tokenized = tokenizer_mini(X_train, return_tensors='tf', padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer_mini(X_test, return_tensors='tf', padding=True, truncation=True, max_length=512)

X_train_tokenized = dict(X_train_tokenized)
X_test_tokenized = dict(X_test_tokenized)
es=EarlyStopping(patience=2, restore_best_weights=True, monitor='loss')


In [ ]:
model_mini.fit(X_train_tokenized,y_train, batch_size=32, epochs=10, callbacks=[es],validation_split=.2)

Epoch 1/10
175/175 [==============================] - 181s 969ms/step - loss: 0.6140 - accuracy: 0.6637 - val_loss: 0.5943 - val_accuracy: 0.6693
Epoch 2/10
175/175 [==============================] - 169s 964ms/step - loss: 0.5164 - accuracy: 0.7514 - val_loss: 0.5406 - val_accuracy: 0.7350
Epoch 3/10
175/175 [==============================] - 182s 1s/step - loss: 0.4405 - accuracy: 0.8000 - val_loss: 0.5319 - val_accuracy: 0.7443
Epoch 4/10
175/175 [==============================] - 179s 1s/step - loss: 0.3622 - accuracy: 0.8388 - val_loss: 0.5360 - val_accuracy: 0.7450
Epoch 5/10
175/175 [==============================] - 183s 1s/step - loss: 0.2992 - accuracy: 0.8745 - val_loss: 0.5412 - val_accuracy: 0.7500
Epoch 6/10
175/175 [==============================] - 181s 1s/step - loss: 0.2285 - accuracy: 0.9080 - val_loss: 0.7607 - val_accuracy: 0.7429
Epoch 7/10
175/175 [==============================] - 185s 1s/step - loss: 0.1859 - accuracy: 0.9268 - val_loss: 0.6137 - val_accuracy: 

In [26]:
y_pred=np.array([[-1,0]])
y_pred[0][0]

-1